In [6]:
from collections import OrderedDict
import numpy as np
import spacy
import codecs

In [7]:
# PICK THE LANGUAGE OF THE CORPUS YOU WISH TO EXAMINE
language = 'de'

In [ ]:
%%time
if language == 'fr':
    print('loading french model')
    nlp = spacy.load("fr_core_news_md")
elif language == 'de':
    print ('loading german model')
    nlp = spacy.load("de_core_news_md")   


loading german model


In [18]:
directory = "../corpus_samples/OCRd/GIP/processed/"
filename = directory + language +'.txt'

In [24]:
with codecs.open(filename, encoding='utf_8') as f:
  lines = f.readlines()
  text = '\n'.join(lines)
print(text)

veroiientlichung Königl Preußische Geodätische Institut FOLGE Nr.69 Jahresbericht Direktor Königliche Geodätische Institut April April Potsdam drucken Stankiewiez Buchdruckerei G.m.b Berlin Exzellenz Königliche Staatsminister Minister geistlich Unterrichts-Angelegenheiten Herrn Trott Solz gehorsamst überreichen Jahresbericht Direktor Königliche Geodätische Institut April April sächlichen ausgeben belaufen Verwendung folgen stellen Tagegelder Reisekosten Stations- beobachtungen außerhalb entfallen Herrn Flotows Aufenthalt Amerika Beobachtung verbunden ausgeben außerordentlich Rechenarbeiten Schreibhilfe verschieden Reise Verwaltung Dotationsfonds Heizmaterial Heizen Reinigen Diensträume Druckkosten dergl Buch Zeitschrift dergl Postgeld dergl Schreibmaterial Instandhaltung Abänderung Anschaffung Untersuchung Instrument aus- wärtige Mechaniker und so weiter mechanische Werkstatt photo- graphische Kammer einschließlich Gehilfenlöhne Material verschieden Mobiliarbeschaffungen insgemein wiss

In [26]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [29]:
nlp.max_length = 1967358
tr4w = TextRank4Keyword()

In [30]:
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(10)

Beobachtung - 58.26677685845676
Herrn - 56.30683123576868
Station - 47.112618262588434
Professor - 45.47470789575291
Herr - 44.95876801098118
Dr. - 40.87448668119915
Arbeit - 32.16898498525927
Instrument - 30.150756833392382
Institut - 29.653912109970367
Berlin - 26.070126486600916
Bestimmung - 25.36822035205879
Fehler - 23.9730048608913


In [43]:
doc = nlp(text)
nouns = [token.text for token in doc if token.is_stop != True and token.is_punct != True and token.pos_ == "NOUN"]
stems = [token.stem for token in doc]

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'stem'

In [39]:
from collections import Counter
word_freq = Counter(nouns)
common_words = word_freq.most_common(20)
common_words

[('Professor', 1262),
 ('Dr.', 1017),
 ('Herrn', 860),
 ('Herr', 758),
 ('Beobachtung', 686),
 ('Institut', 604),
 ('Station', 599),
 ('Nummer', 533),
 ('Arbeit', 450),
 ('Bericht', 326),
 ('Ergebnis', 322),
 ('Bestimmung', 299),
 ('Messung', 299),
 ('Instrument', 298),
 ('Apparat', 273),
 ('Seite', 259),
 ('Veröffentlichung', 241),
 ('Untersuchung', 237),
 ('Netz', 236),
 ('Länge', 224)]

In [42]:
counter = 0
for noun in nouns:
    if(noun == 'Triangulation'):
        counter += 1
counter

25